<a href="https://colab.research.google.com/github/meafekih/links_kaggle/blob/master/Detect_AI_Generated_(_Clustering_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import colors
import datetime
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting library
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings("ignore")

palette = ["#a7e149","#e149a7", "#c393f4", "#d683f2", "#dd9e9e", "#fa91aa", "#fc9e9e", "#49e183", "#aa91aa", "#ff93f4"]


In [11]:
import nltk
from nltk.tokenize import sent_tokenize
from rouge import Rouge
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
!pip install rouge
!pip install textstat

ModuleNotFoundError: ignored

In [9]:
train=pd.read_csv("/content/train_essays.csv")
train_prompts=pd.read_csv("/content/train_prompts.csv")

In [4]:
train0 = train[(train['prompt_id']==0) ]
train1 = train[(train['prompt_id']==1) ]

In [5]:
train = train0.copy()
prompt= train_prompts.iloc[0][3]
instructions= train_prompts.iloc[0][2]

In [6]:
train = train1.copy()
prompt= train_prompts.iloc[1][3]
instructions= train_prompts.iloc[1][2]

In [7]:
def custom_tokenizer(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    tagged_tokens = pos_tag(filtered_tokens)
    lst = ['VBN', 'VB', 'JJ', 'NNS', 'NN']
    #lst = ['NN']
    selected_tokens = [word for word, pos in tagged_tokens if pos in lst]
    return selected_tokens


def calculate_tfidf(row):
    text = row['text']
    prompt_and_text = [prompt, text]
    # Initialize TfidfVectorizer with custom tokenizer
    tfidf_vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
    # Fit and transform the combined prompt and text
    tfidf_matrix = tfidf_vectorizer.fit_transform(prompt_and_text)
    # Calculate cosine similarity between the prompt and the text
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    # Return the cosine similarity value
    return cosine_sim[0][0]


train['cosine_similarity'] = train.apply(calculate_tfidf, axis=1)

In [13]:
def calculate_rouge_params(text, reference):
    rouge = Rouge()
    rouge_scores = rouge.get_scores(text, reference)[0]
    rouge_1_pr, rouge_1_re, rouge_1_f1 = rouge_scores['rouge-1']['p'], rouge_scores['rouge-1']['r'], rouge_scores['rouge-1']['f']
    rouge_2_pr, rouge_2_re, rouge_2_f1 = rouge_scores['rouge-2']['p'], rouge_scores['rouge-2']['r'], rouge_scores['rouge-2']['f']
    rouge_l_pr, rouge_l_re, rouge_l_f1 = rouge_scores['rouge-l']['p'], rouge_scores['rouge-l']['r'], rouge_scores['rouge-l']['f']
    return rouge_1_pr, rouge_1_re, rouge_1_f1, rouge_2_pr, rouge_2_re, rouge_2_f1, rouge_l_pr, rouge_l_re, rouge_l_f1

train[['rouge_1_pr', 'rouge_1_re', 'rouge_1_f1', 'rouge_2_pr',
        'rouge_2_re', 'rouge_2_f1', 'rouge_l_pr', 'rouge_l_re', 'rouge_l_f1'
        ]] = train['text'].apply(lambda x: pd.Series(calculate_rouge_params(x, prompt)))


In [14]:

train[['irouge_1_pr', 'irouge_1_re', 'irouge_1_f1', 'irouge_2_pr',
        'irouge_2_re', 'irouge_2_f1', 'irouge_l_pr', 'irouge_l_re', 'irouge_l_f1'
        ]] = train['text'].apply(lambda x: pd.Series(calculate_rouge_params(x, instructions)))

In [15]:
train.head().T

,0,1,2,3,4
id,0059830c,005db917,008f63e3,00940276,00c39458
prompt_id,0,0,0,0,0
text,Cars. Cars have been around since they became ...,Transportation is a large necessity in most co...,"""America's love affair with it's vehicles seem...",How often do you ride in a car? Do you drive a...,Cars are a wonderful thing. They are perhaps o...
generated,0,0,0,0,0
rouge_1_pr,0.289037,0.310345,0.258953,0.260546,0.280423
rouge_1_re,0.113134,0.105332,0.122237,0.136541,0.137841
rouge_1_f1,0.162617,0.157282,0.166078,0.179181,0.18483
rouge_2_pr,0.055118,0.053118,0.063063,0.045181,0.055629
rouge_2_re,0.016949,0.013923,0.025424,0.01816,0.025424
rouge_2_f1,0.025926,0.022062,0.036238,0.025907,0.034898


In [16]:
from textstat import flesch_reading_ease,flesch_kincaid_grade
from nltk.translate.bleu_score import sentence_bleu
import nltk
import textstat
import string
from nltk.corpus import stopwords

# train['fre'] = train['text'].apply(lambda x : flesch_reading_ease(x))
# train['fkg'] = train['text'].apply(lambda x : flesch_kincaid_grade(x))

# Assuming you have a DataFrame named 'train' with a 'text' column
train['bleu_score'] = train['text'].apply(lambda x: sentence_bleu([prompt.split()], x.split()))



In [17]:
train['word_count'] = train['text'].apply(lambda x: len(x.split()))
train['avg_word_length'] = train['text'].apply(lambda x: sum(len(word) for word in x.split()) / len(x.split()))
train['sentence_count'] = train['text'].apply(lambda x: len(nltk.sent_tokenize(x)))
train['punctuation_count'] = train['text'].apply(lambda x: len([char for char in x if char in string.punctuation]))
train['flesch_reading_ease'] = train['text'].apply(lambda x: textstat.flesch_reading_ease(x))
train['stopword_count'] = train['text'].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))



In [18]:
import spacy

nlp = spacy.load('en_core_web_sm')
train['named_entities'] = train['text'].apply(lambda x: [(ent.text, ent.label_) for ent in nlp(x).ents])

train['pos_tags'] = train['text'].apply(lambda x: [(token.text, token.pos_) for token in nlp(x)])
train['dependency_tree'] = train['text'].apply(lambda x: [(token.text, token.dep_, token.head.text) for token in nlp(x)])

# from gensim.models import Word2Vec
# model = Word2Vec.load('word2vec_model.bin')
# train['word_embeddings'] = train['text'].apply(lambda x: [model[word] for word in x.split() if word in model])

from textblob import TextBlob
train['sentiment'] = train['text'].apply(lambda x: TextBlob(x).sentiment.polarity)



from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(train['text'])
lda = LatentDirichletAllocation(n_components=5, random_state=42)
topic_distribution = lda.fit_transform(X)

# Create new columns for each component
for i in range(lda.n_components):
    train[f'topic_component_{i+1}'] = topic_distribution[:, i]


In [33]:
data = train.copy()
data_cols = data[[
    'rouge_1_pr', 'rouge_1_re', 'rouge_1_f1',
    'rouge_2_pr', 'rouge_2_re','rouge_2_f1',
    'rouge_l_pr', 'rouge_l_re', 'rouge_l_f1',

    'irouge_1_pr', 'irouge_1_re', 'irouge_1_f1',
    'irouge_2_pr', 'irouge_2_re','irouge_2_f1',
    'irouge_l_pr', 'irouge_l_re', 'irouge_l_f1',
    # 'fre','fkg'
    'bleu_score',

    'word_count', 'avg_word_length', 'sentence_count','punctuation_count',
    'flesch_reading_ease', 'stopword_count',

    'sentiment', 'topic_component_1' , 'topic_component_2' ,
    'topic_component_3','topic_component_4' , 'topic_component_5',

    'compound_sentiment',  'noun_percentage', 'verb_percentage', 'adjective_percentage', 'adverb_percentage'
            ]]
essay_features = data_cols.copy()
# K-Means clustering
num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(essay_features)

# Add cluster labels and names to the original dataset
data['cluster_label'] = kmeans.labels_
data['cluster_name'] = data['cluster_label'].map({0: 'Human', 1: 'LLM'})


In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit and transform the DataFrame
essay_features = pd.DataFrame(scaler.fit_transform(essay_features), columns=essay_features.columns)


In [38]:
# # Normalize the data
# scaler = StandardScaler()
# essay_features = pd.DataFrame(scaler.fit_transform(essay_features), columns=essay_features.columns)


In [39]:

# Apply PCA for 3D visualization
pca = PCA(n_components=3)
essay_features_3d = pca.fit_transform(essay_features)

essay_features_3d = np.concatenate((essay_features_3d, data['generated'].values.reshape(-1, 1)), axis=1)
essay_features_3d = np.concatenate((essay_features_3d, data['id'].values.reshape(-1, 1)), axis=1)
essay_features_3d

array([[-3.6778477056738526, 0.25761880405016063, 0.6868450589451099,
        0.0, '0059830c'],
       [-3.046288197129241, -0.8086613086253621, 1.6379396602734093, 0.0,
        '005db917'],
       [-4.117378308046179, 2.8419115524535568, 1.467643689063963, 0.0,
        '008f63e3'],
       ...,
       [-3.8310446564169824, 2.601834395837972, 1.0512948308017998, 0.0,
        'ffa247e0'],
       [-3.3201840817511754, -0.468115581454021, -1.527990970429831, 0.0,
        'ffc237e9'],
       [-2.6438989273703157, -1.527853330457458, 1.713269508127817, 0.0,
        'ffe1ca0d']], dtype=object)

In [40]:
import plotly.express as px

# Scatter plot for all rows
#color=essay_features_3d[:, 3],

scatter = px.scatter_3d(
    x=essay_features_3d[:, 0],y=essay_features_3d[:, 1],z=essay_features_3d[:, 2],
    #color=kmeans.labels_,
    color=essay_features_3d[:, 3],
    labels={'color': 'Cluster'},title='K-Means Clustering - Human vs. LLM (3D PCA)',
    template="simple_white",opacity=0.8,
)

# Update layout
scatter.update_layout(scene=dict(
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    zaxis_title='Principal Component 3'))

# Show plots
scatter.show()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [31]:
from nltk.sentiment import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
train['compound_sentiment'] = train['text'].apply(lambda x: sid.polarity_scores(x)['compound'])


In [32]:

def calculate_pos_percentage(text, pos_tag_to_count):
    words = word_tokenize(text)
    pos_tags = pos_tag(words)
    pos_count = sum(1 for _, tag in pos_tags if tag.startswith(pos_tag_to_count))
    total_words = len(words)
    pos_percentage = (pos_count / total_words) * 100 if total_words > 0 else 0
    return pos_percentage

train['noun_percentage'] = train['text'].apply(lambda x: calculate_pos_percentage(x, 'N'))
train['verb_percentage'] = train['text'].apply(lambda x: calculate_pos_percentage(x, 'V'))
train['adjective_percentage'] = train['text'].apply(lambda x: calculate_pos_percentage(x, 'J'))
train['adverb_percentage'] = train['text'].apply(lambda x: calculate_pos_percentage(x, 'R'))

In [ ]:
# print(train_prompts.iloc[0][1])
# train_prompts.iloc[0][2]

In [ ]:
# print(train_prompts.iloc[1][1])
# train_prompts.iloc[1][2]

In [ ]:
# human0= train[(train['generated']==0) & (train['prompt_id']==0) ]
# human1= train[(train['generated']==0) & (train['prompt_id']==1) ]
# ai0 =   train[(train['generated']==1) & (train['prompt_id']==0) ]
# ai1 =   train[(train['generated']==1) & (train['prompt_id']==1) ]


# len(human0), len(human1), len(ai0), len(ai1)

# <p id="1" style="background-color:#fef0e9;font-family:newtimeroman;font-size:150%;color:#AF4343;text-align:center;border-radius:15px 50px;padding:7px;border:solid 3px #AF4343;">The Three AI text generated</p>

In [ ]:
# # AI responce prompt 0
# ai0.iloc[0][2]

In [ ]:
# # AI responce prompt 1
# ai1.iloc[0][2]

In [ ]:
# # AI responce prompt 1
# ai1.iloc[1][2]

In [ ]:
# responseAI0 = train[(train['generated']==1) & (train['prompt_id']==0)]
# responseAI0.iloc[0][2]

In [ ]:
#train0 = train1.copy()

# <p id="1" style="background-color:#fef0e9;font-family:newtimeroman;font-size:150%;color:#AF4343;text-align:center;border-radius:15px 50px;padding:7px;border:solid 3px #AF4343;">TFIDF prompts</p>

In [ ]:
prompt0= train_prompts.iloc[0][3]

In [ ]:
prompt0= train_prompts.iloc[1][3]
train0 = train1.copy()

In [ ]:

def calculate_jaccard_similarity(row):
    text = row['text']
    prompt_and_text = [prompt0, text]

    # Initialize CountVectorizer
    count_vectorizer = CountVectorizer()

    # Fit and transform the combined prompt and text
    count_matrix = count_vectorizer.fit_transform(prompt_and_text)

    # Calculate Jaccard similarity between the prompt and the text
    intersection = count_matrix[0].multiply(count_matrix[1]).sum()
    union = count_matrix[0].sum() + count_matrix[1].sum() - intersection
    jaccard_similarity = intersection / union

    # Return the Jaccard similarity value
    return jaccard_similarity


train0['jaccard_similarity'] = train0.apply(calculate_jaccard_similarity, axis=1)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer

def calculate_euclidean_distance(row):
    text = row['text']
    prompt_and_text = [prompt0, text]

    # Initialize TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the combined prompt and text
    tfidf_matrix = tfidf_vectorizer.fit_transform(prompt_and_text)

    # Calculate Euclidean distance between the prompt and the text
    euclidean_dist = euclidean_distances(tfidf_matrix[0], tfidf_matrix[1])

    # Return the Euclidean distance value
    return euclidean_dist[0][0]


train0['euclidean_distance'] = train0.apply(calculate_euclidean_distance, axis=1)

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.feature_extraction.text import TfidfVectorizer

def calculate_manhattan_distance(row):
    text = row['text']
    prompt_and_text = [prompt0, text]

    # Initialize TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the combined prompt and text
    tfidf_matrix = tfidf_vectorizer.fit_transform(prompt_and_text)

    # Calculate Manhattan distance between the prompt and the text
    manhattan_dist = manhattan_distances(tfidf_matrix[0], tfidf_matrix[1])

    # Return the Manhattan distance value
    return manhattan_dist[0][0]

train0['manhattan_distance'] = train0.apply(calculate_manhattan_distance, axis=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def calculate_dice_similarity(row):
    text = row['text']
    prompt_and_text = [prompt0, text]

    # Initialize CountVectorizer
    count_vectorizer = CountVectorizer()

    # Fit and transform the combined prompt and text
    count_matrix = count_vectorizer.fit_transform(prompt_and_text)

    # Calculate Dice similarity between the prompt and the text
    intersection = count_matrix[0].multiply(count_matrix[1]).sum()
    dice_similarity = 2 * intersection / (count_matrix[0].sum() + count_matrix[1].sum())

    # Return the Dice similarity value
    return dice_similarity

train0['dice_similarity'] = train0.apply(calculate_dice_similarity, axis=1)

In [ ]:
import numpy as np
from scipy.spatial import distance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

def calculate_jensen_shannon_divergence(row):
    text = row['text']
    prompt_and_text = [prompt0, text]

    # Initialize CountVectorizer
    count_vectorizer = CountVectorizer()

    # Fit and transform the combined prompt and text
    count_matrix = count_vectorizer.fit_transform(prompt_and_text)

    # Normalize count matrix
    norm_count_matrix = normalize(count_matrix, norm='l1', axis=1)

    # Calculate Jensen-Shannon Divergence
    p = norm_count_matrix[0].toarray().ravel()
    q = norm_count_matrix[1].toarray().ravel()
    m = 0.5 * (p + q)
    js_divergence = 0.5 * (distance.jensenshannon(p, m) + distance.jensenshannon(q, m))

    # Return the Jensen-Shannon Divergence value
    return js_divergence

train0['jensen_shannon_divergence'] = train0.apply(calculate_jensen_shannon_divergence, axis=1)

In [ ]:
train0.head().T
#train0[train0['id']=="5b57999b"].iloc[0][2]

,5,6,8,9,10
id,00da8c32,011dc2bc,01c6e176,0202ddf9,020a5d6d
prompt_id,1,1,1,1,1
text,The electrol college system is an unfair syste...,"Dear state senator, It is the utmost respect t...","""It's official: The electoral college is unfai...",The Electoral College has been kept for centur...,"Dear senator, Retain the Electoral College. Th..."
generated,0,0,0,0,0
Mean_Words,26.454545,17.28,18.84,23.708333,25.285714
Median_Words,19.5,15.0,16.0,23.5,20.0
cosine_similarity,0.565721,0.685427,0.646569,0.710208,0.694767
jaccard_similarity,-1.203314,-1.176945,-1.328181,-1.179725,-1.278418
euclidean_distance,0.603599,0.474653,0.711313,0.482524,0.517545
manhattan_distance,9.520242,8.291137,11.006962,8.409343,8.405009


In [ ]:
from rouge import Rouge
def calculate_rouge_params(text):
    def calculate_rouge(hypothesis, reference):
        rouge = Rouge()
        scores = rouge.get_scores(hypothesis, reference)
        return scores
    rouge_scores = calculate_rouge(text, prompt0)

    # Accessing different ROUGE scores
    rouge_1_pr = rouge_scores[0]['rouge-1']['p']
    rouge_1_re = rouge_scores[0]['rouge-1']['r']
    rouge_1_f1 = rouge_scores[0]['rouge-1']['f']

    rouge_2_pr = rouge_scores[0]['rouge-2']['p']
    rouge_2_re = rouge_scores[0]['rouge-2']['r']
    rouge_2_f1 = rouge_scores[0]['rouge-2']['f']

    rouge_l_pr = rouge_scores[0]['rouge-l']['p']
    rouge_l_re = rouge_scores[0]['rouge-l']['r']
    rouge_l_f1 = rouge_scores[0]['rouge-l']['f']
    return rouge_1_pr, rouge_1_re, rouge_1_f1, rouge_2_pr, rouge_2_re, rouge_2_f1, rouge_l_pr, rouge_l_re, rouge_l_f1

train0[['rouge_1_pr', 'rouge_1_re', 'rouge_1_f1', 'rouge_2_pr', 'rouge_2_re', 'rouge_2_f1',
       'rouge_l_pr', 'rouge_l_re', 'rouge_l_f1']] = train0['text'].apply(lambda x: pd.Series(calculate_rouge_params(x)))

In [ ]:
train0.head().T

,5,6,8,9,10
id,00da8c32,011dc2bc,01c6e176,0202ddf9,020a5d6d
prompt_id,1,1,1,1,1
text,The electrol college system is an unfair syste...,"Dear state senator, It is the utmost respect t...","""It's official: The electoral college is unfai...",The Electoral College has been kept for centur...,"Dear senator, Retain the Electoral College. Th..."
generated,0,0,0,0,0
Mean_Words,26.454545,17.28,18.84,23.708333,25.285714
Median_Words,19.5,15.0,16.0,23.5,20.0
cosine_similarity,0.565721,0.685427,0.646569,0.710208,0.694767
jaccard_similarity,-1.203314,-1.176945,-1.328181,-1.179725,-1.278418
euclidean_distance,0.603599,0.474653,0.711313,0.482524,0.517545
manhattan_distance,9.520242,8.291137,11.006962,8.409343,8.405009


In [ ]:
essay_features_3d

array([[-0.13791460839990072, -0.030744064956684282,
        -0.05573569104792567, 0.0, '00da8c32'],
       [-0.027219269998412485, -0.043182716800910646,
        -0.0004989543904592907, 0.0, '011dc2bc'],
       [0.07411475130835309, -0.006367545310483466, 0.008467099950267046,
        0.0, '01c6e176'],
       ...,
       [0.1755754263872074, 0.03235681135940331, -0.012349121738512202,
        0.0, 'fcd93e2d'],
       [0.22541627699048072, 0.0005127272090163797,
        -0.006805016926093791, 0.0, 'fcfe84cb'],
       [0.08870807049257959, -0.026397619918889864,
        -0.009990244621344062, 0.0, 'fe6ff9a5']], dtype=object)

In [ ]:
'Mean_Words','Median_Words',
             'cosine_similarity','jaccard_similarity',
             'euclidean_distance','manhattan_distance',
            "dice_similarity","jensen_shannon_divergence",

In [ ]:
data = train0.copy()
test = data[[

'rouge_1_pr', 'rouge_1_re', 'rouge_1_f1', 'rouge_2_pr', 'rouge_2_re',
'rouge_2_f1','rouge_l_pr', 'rouge_l_re', 'rouge_l_f1'
            ]]
essay_features = test.copy()
# K-Means clustering
num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(essay_features)

# Add cluster labels and names to the original dataset
data['cluster_label'] = kmeans.labels_
data['cluster_name'] = data['cluster_label'].map({0: 'Human', 1: 'LLM'})

# Apply PCA for 3D visualization
pca = PCA(n_components=3)
essay_features_3d = pca.fit_transform(essay_features)


essay_features_3d = np.concatenate((essay_features_3d, data['generated'].values.reshape(-1, 1)), axis=1)
essay_features_3d = np.concatenate((essay_features_3d, data['id'].values.reshape(-1, 1)), axis=1)

In [ ]:
import plotly.express as px

# Scatter plot for all rows
#color=essay_features_3d[:, 3],

scatter = px.scatter_3d(
    x=essay_features_3d[:, 0],y=essay_features_3d[:, 1],z=essay_features_3d[:, 3],
    color=kmeans.labels_,
    labels={'color': 'Cluster'},title='K-Means Clustering - Human vs. LLM (3D PCA)',
    template="simple_white",opacity=0.8,
)

# Update layout
scatter.update_layout(scene=dict(
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    zaxis_title='Principal Component 3'))

# Show plots
scatter.show()


NN (Noun, singular): Represents a single, standalone noun.

Example: cat, dog, book
NNS (Noun, plural): Represents a plural noun.

Example: cats, dogs, books
JJ (Adjective): Represents an adjective, which describes or modifies a noun.

Example: big, happy, green
VB (Verb, base form): Represents a base form of a verb.

Example: run, eat, play
VBD (Verb, past tense): Represents a past tense verb.

Example: ran, ate, played
VBG (Verb, gerund/present participle): Represents a verb form ending in '-ing'.

Example: running, eating, playing
VBN (Verb, past participle): Represents a past participle form of a verb.

Example: written, eaten, played
RB (Adverb): Represents an adverb, which describes or modifies a verb, adjective, or other adverb.

Example: quickly, very, well
IN (Preposition or subordinating conjunction): Represents a preposition or a subordinating conjunction.

Example: in, on, under

<p style="background-color:#e6daf9;font-family:newtimeroman;font-size:170%;color:#7541ca;text-align:center;border-radius:10px 70px;padding:7px;border:solid 3px #4e2b87;">If my notebook has imparted valuable insights to you, your upvote is sincerely appreciated.
</p>

In [ ]:
from rouge import Rouge

def calculate_rouge(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return scores

# Example Usage:
reference_summary = "This is a reference summary."
system_summary = "This is a system-generated summary."

rouge_scores = calculate_rouge(system_summary, reference_summary)

# Accessing different ROUGE scores
rouge_1_precision = rouge_scores[0]['rouge-1']['p']
rouge_1_recall = rouge_scores[0]['rouge-1']['r']
rouge_1_f1 = rouge_scores[0]['rouge-1']['f']

rouge_2_precision = rouge_scores[0]['rouge-2']['p']
rouge_2_recall = rouge_scores[0]['rouge-2']['r']
rouge_2_f1 = rouge_scores[0]['rouge-2']['f']

rouge_l_precision = rouge_scores[0]['rouge-l']['p']
rouge_l_recall = rouge_scores[0]['rouge-l']['r']
rouge_l_f1 = rouge_scores[0]['rouge-l']['f']


In [ ]:
rouge_scores

[{'rouge-1': {'r': 0.8, 'p': 0.8, 'f': 0.7999999950000002},
  'rouge-2': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001},
  'rouge-l': {'r': 0.8, 'p': 0.8, 'f': 0.7999999950000002}}]

In [ ]:

# rouge_w_precision = rouge_scores[0]['rouge-w']['p']
# rouge_w_recall = rouge_scores[0]['rouge-w']['r']
# rouge_w_f1 = rouge_scores[0]['rouge-w']['f']

# rouge_su_precision = rouge_scores[0]['rouge-su']['p']
# rouge_su_recall = rouge_scores[0]['rouge-su']['r']
# rouge_su_f1 = rouge_scores[0]['rouge-su']['f']

# rouge_p_precision = rouge_scores[0]['rouge-p']['p']
# rouge_p_recall = rouge_scores[0]['rouge-p']['r']
# rouge_p_f1 = rouge_scores[0]['rouge-p']['f']

# Print ROUGE scores
print("ROUGE-1 Precision:", rouge_1_precision)
print("ROUGE-1 Recall:", rouge_1_recall)
print("ROUGE-1 F1 Score:", rouge_1_f1)

print("\nROUGE-2 Precision:", rouge_2_precision)
print("ROUGE-2 Recall:", rouge_2_recall)
print("ROUGE-2 F1 Score:", rouge_2_f1)

print("\nROUGE-L Precision:", rouge_l_precision)
print("ROUGE-L Recall:", rouge_l_recall)
print("ROUGE-L F1 Score:", rouge_l_f1)

# print("\nROUGE-W Precision:", rouge_w_precision)
# print("ROUGE-W Recall:", rouge_w_recall)
# print("ROUGE-W F1 Score:", rouge_w_f1)

# print("\nROUGE-SU Precision:", rouge_su_precision)
# print("ROUGE-SU Recall:", rouge_su_recall)
# print("ROUGE-SU F1 Score:", rouge_su_f1)

# print("\nROUGE-P Precision:", rouge_p_precision)
# print("ROUGE-P Recall:", rouge_p_recall)
# print("ROUGE-P F1 Score:", rouge_p_f1)


ROUGE-1 Precision: 0.8
ROUGE-1 Recall: 0.8
ROUGE-1 F1 Score: 0.7999999950000002

ROUGE-2 Precision: 0.5
ROUGE-2 Recall: 0.5
ROUGE-2 F1 Score: 0.4999999950000001

ROUGE-L Precision: 0.8
ROUGE-L Recall: 0.8
ROUGE-L F1 Score: 0.7999999950000002


In [ ]:
def Calculate_Words(text):
    sentences = sent_tokenize(text)
    #median
    words_per_sentence = [len(sentence.split()) for sentence in sentences]
    median_words = np.median(words_per_sentence)
    #mean
    total_words = 0
    for sentence in sentences:
        words = sentence.split()
        total_words += len(words)
    mean_words = total_words / len(sentences)
    return mean_words,median_words

#train[['Mean_Words', 'Median_Words']] = train['text'].apply(Calculate_Words)
train[['Mean_Words', 'Median_Words']] = train['text'].apply(lambda x: pd.Series(Calculate_Words(x)))
